# 458 - Permutations of Project

## Problem Statement

Consider the alphabet $A$ made out of the letters of the word "$\text{project}$": $A=\{\text c,\text e,\text j,\text o,\text p,\text r,\text t\}$.<br>
Let $T(n)$ be the number of strings of length $n$ consisting of letters from $A$ that do not have a substring that is one of the $5040$ permutations of "$\text{project}$".

$T(7)=7^7-7!=818503$.

Find $T(10^{12})$. Give the last $9$ digits of your answer.

## Solution

We define 8 states representing the number of consecutive distinct letters we currently have. Initially, we do not have any letter so we start in state 0. From state zero, we are guaranteed to go to state 1. This is because the first letter in the string cannot be preceded by any letter. Then going through the string, we count the number of consecutive letters we have and this is the current state. Say we have 3 letters P, R and O in this order. There are 4 ways to go to the next state (picking J, E, C or T). There is one way to go back to state 1, if the next letter is a O. There is one way to go back to state 2, if the next letter is R. And one way to stay in state 3, if the next letter is a P. If at any point we reach state 7, meaning the last seven letters are a permutation of project, we stay in this state indefinitely, indicating there is at least one permutation of project in the string.

We can represent the state transitions with an $8 \times 8$ matrix, $A$, where $A_{i, j}$ which indicates the number of ways to go from state $i$ to state $j$. The matrix is

\begin{equation}
    A = \begin{bmatrix}
    0 & 7 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 1 & 6 & 0 & 0 & 0 & 0 & 0 \\
    0 & 1 & 1 & 5 & 0 & 0 & 0 & 0 \\
    0 & 1 & 1 & 1 & 4 & 0 & 0 & 0 \\
    0 & 1 & 1 & 1 & 1 & 3 & 0 & 0 \\
    0 & 1 & 1 & 1 & 1 & 1 & 2 & 0 \\
    0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 7
    \end{bmatrix}.
\end{equation}

For a string of length $n = 10^{12}$, we need $A^{10^{12}}$. The number of ways will be very large and we only need the last 9 digits. Therefore, we keep all the matrix entries modulo $10^9$. In order to do the exponentiation is a reasonable time, we use the exponentiation by squaring technique which is based on the observation that

\begin{equation}
    x^n = \begin{cases}
        x(x^2)^\frac{n - 1}{2}, & \text{if } n \text{ is odd} \\
        (x^2)^{\frac{n}{2}}, & \text{if } n \text{ is even}
        
    \end{cases}.
\end{equation}

This allows to compute large powers very fast.

The (last digits of the) number of ways to go from state 0 to state 8 in the string is $A^n_{0, 7}$. This is the number of strings containing a permutation of project at least once. Therefore, we need to subtract this from $7^{10^{12}}$ and take modulo $10^9$ to get the final answer.

In [1]:
import numpy as np

def matrix_modular_exponentiation(matrix, power, mod):
    # Ensure the matrix is in integer format for modular operations
    matrix = np.array(matrix, dtype=np.int64)
    # Start with the identity matrix of the same size as the input matrix
    result = np.eye(matrix.shape[0], dtype=np.int64)

    # Apply exponentiation by squaring technique
    while power > 0:
        if power % 2 == 1:
            result = np.dot(result, matrix) % mod
        matrix = np.dot(matrix, matrix) % mod
        power //= 2

    return result


def modular_exponentiation(base, exponent, mod):
    result = 1
    current_base = base % mod  # Reduce base modulo first to keep numbers small

    while exponent > 0:
        if exponent % 2 == 1:  # If exponent is odd, multiply the current result by the base
            result = (result * current_base) % mod
        current_base = (current_base * current_base) % mod  # Square the base
        exponent //= 2  # Divide the exponent by 2

    return result

In [2]:
exponent = 10**12
mod = 10**9

A = np.array([
    [0, 7, 0, 0, 0, 0, 0, 0],
    [0, 1, 6, 0, 0, 0, 0, 0],
    [0, 1, 1, 5, 0, 0, 0, 0],
    [0, 1, 1, 1, 4, 0, 0, 0],
    [0, 1, 1, 1, 1, 3, 0, 0],
    [0, 1, 1, 1, 1, 1, 2, 0],
    [0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 7]
])

power = 10**12
mod = 10**9  # We want the last 10 digits

# Compute the matrix power modulo 10^10
matrix = matrix_modular_exponentiation(A, power, mod)

(modular_exponentiation(7, power, mod) - matrix[0, -1]) % mod

423341841